# Linear Regression 02 — Optimization & Training from Scratch  
**Deccan AI School (Premium Bootcamp)** — Working Professionals (IT/Software)

**Goal:** Implement training like an ML engineer:
- Build cost function
- Derive gradients
- Implement gradient descent (batch + mini-batch)
- Understand learning rate, scaling, divergence
- Visualize loss landscape and convergence

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 5)

## 1) Setup: a slightly noisy dataset

We generate synthetic data:
\[
y = 2.5 + 1.7x + \epsilon
\]

Why synthetic?
- In early learning, you want truth you control.
- You can verify if your algorithm recovers the real parameters.

In [ ]:
rng = np.random.default_rng(42)
x = np.linspace(0, 10, 80)
noise = rng.normal(0, 1.0, size=len(x))
y = 2.5 + 1.7*x + noise

plt.scatter(x, y, s=20)
plt.title("Synthetic data (known ground truth)")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

## 2) Design matrix (bias trick)

Instead of writing \(\theta_0 + \theta_1 x\) repeatedly,
we build:

\[
X = \begin{bmatrix}
1 & x_1 \\
1 & x_2 \\
\vdots & \vdots \\
1 & x_n
\end{bmatrix}
,\quad
\theta = \begin{bmatrix}\theta_0 \\ \theta_1\end{bmatrix}
\]

Then predictions:
\[
\hat{y} = X\theta
\]

In [ ]:
X = np.c_[np.ones_like(x), x]  # (n,2)
y = y.reshape(-1, 1)          # (n,1)

## 3) Cost function (MSE) in matrix form

\[
J(\theta) = \frac{1}{n} \|X\theta - y\|^2
\]

We’ll implement it, then later use it for debugging training.

In [ ]:
def mse_cost(X, y, theta):
    errors = X @ theta - y
    return float(np.mean(errors**2))

theta_test = np.array([[0.0],[0.0]])
mse_cost(X, y, theta_test)

## 4) Gradient derivation (core)

We want gradient w.r.t theta:

\[
\nabla_\theta J(\theta) = \frac{2}{n}X^T(X\theta - y)
\]

This is *the* most important formula for gradient descent.
If you understand this once, you can implement many models later.

In [ ]:
def mse_gradient(X, y, theta):
    n = X.shape[0]
    return (2/n) * (X.T @ (X @ theta - y))

mse_gradient(X, y, theta_test)

## 5) Batch Gradient Descent

Update rule:
\[
\theta := \theta - \alpha \nabla_\theta J(\theta)
\]

Where:
- \(\alpha\) is learning rate

**Common engineer mistake:** too high learning rate → divergence.

In [ ]:
def batch_gd(X, y, alpha=0.05, iters=500):
    theta = np.zeros((X.shape[1], 1))
    history = []
    for i in range(iters):
        grad = mse_gradient(X, y, theta)
        theta = theta - alpha * grad
        history.append(mse_cost(X, y, theta))
    return theta, history

theta_gd, hist = batch_gd(X, y, alpha=0.05, iters=400)
theta_gd.ravel(), hist[-1]

In [ ]:
plt.plot(hist)
plt.title("Batch GD: Loss convergence")
plt.xlabel("Iteration")
plt.ylabel("MSE")
plt.grid(True)
plt.show()

## 6) Compare with closed-form (Normal Equation)

Closed-form:
\[
\theta = (X^TX)^{-1}X^Ty
\]

This is OLS solution.

**Engineering note:** For huge datasets, matrix inversion can be expensive or unstable,
so GD / SGD is preferred.

In [ ]:
theta_ols = np.linalg.inv(X.T @ X) @ (X.T @ y)
print("GD theta:", theta_gd.ravel())
print("OLS theta:", theta_ols.ravel())

## 7) Loss landscape intuition (2D parameter search)

We’ll compute loss on a grid of (\(\theta_0,\theta_1\)) values to see the bowl shape.
This explains:
- why GD works (convex surface)
- why there is a unique global minimum (for full rank X)

In [ ]:
t0_vals = np.linspace(-1, 6, 80)
t1_vals = np.linspace(0, 3, 80)

Z = np.zeros((len(t0_vals), len(t1_vals)))
for i, t0 in enumerate(t0_vals):
    for j, t1 in enumerate(t1_vals):
        theta = np.array([[t0],[t1]])
        Z[i, j] = mse_cost(X, y, theta)

plt.contour(t1_vals, t0_vals, Z, levels=30)
plt.scatter([theta_gd[1,0]], [theta_gd[0,0]], marker="x", s=80)
plt.title("Loss contour (theta1 on x-axis, theta0 on y-axis)")
plt.xlabel("theta1 (slope)")
plt.ylabel("theta0 (intercept)")
plt.grid(True)
plt.show()

## 8) Learning rate experiment (must do)

We will train with multiple learning rates and compare convergence.
This is a very common interview + production debugging question.

In [ ]:
lrs = [0.001, 0.01, 0.05, 0.2]
histories = {}

for lr in lrs:
    _, h = batch_gd(X, y, alpha=lr, iters=200)
    histories[lr] = h

for lr, h in histories.items():
    plt.plot(h, label=f"alpha={lr}")

plt.title("Learning rate comparison")
plt.xlabel("Iteration")
plt.ylabel("MSE")
plt.grid(True)
plt.legend()
plt.show()

## 9) Feature scaling: why it matters (preview)

When features have very different scales, gradients become messy and GD converges slowly.

We'll simulate it by creating a second feature with larger scale in Notebook 03,
but here’s the key message:

> If you use gradient descent, scaling is not optional.

## 10) Mini-batch gradient descent (practical)

In real life:
- dataset is large
- we don’t compute gradient on the full dataset each step

Mini-batch combines benefits of batch and stochastic.

In [ ]:
def minibatch_gd(X, y, alpha=0.05, iters=200, batch_size=16, seed=42):
    rng = np.random.default_rng(seed)
    theta = np.zeros((X.shape[1], 1))
    history = []
    n = X.shape[0]
    for i in range(iters):
        idx = rng.choice(n, size=batch_size, replace=False)
        Xb, yb = X[idx], y[idx]
        grad = mse_gradient(Xb, yb, theta)
        theta = theta - alpha * grad
        history.append(mse_cost(X, y, theta))  # track full loss
    return theta, history

theta_mbgd, hist_mbgd = minibatch_gd(X, y, alpha=0.05, iters=400, batch_size=16)

plt.plot(hist, label="Batch GD")
plt.plot(hist_mbgd, label="Mini-batch GD")
plt.title("Batch vs Mini-batch convergence (full loss tracked)")
plt.xlabel("Iteration")
plt.ylabel("MSE")
plt.grid(True)
plt.legend()
plt.show()

theta_mbgd.ravel(), theta_ols.ravel()

## 11) Mini Assignment (Interview-style)

1. Set learning rate = 0.5 and observe what happens.
2. Explain divergence in 3 lines.
3. Implement an early stopping rule:
   - stop if improvement < 1e-6 for 10 consecutive iterations.

**Deliverable:** code cell + markdown explanation.